In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U xformers


In [3]:
import transformers

from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig)
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from peft import (
    LoraConfig,
    PeftConfig,PeftModel)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
def LoadModel1(PEFT_MODEL=''):
      bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16)
      config = PeftConfig.from_pretrained(PEFT_MODEL)
      model = AutoModelForCausalLM.from_pretrained(
          config.base_model_name_or_path,
          return_dict=True,
          quantization_config=bnb_config,
          device_map="auto",
          trust_remote_code=True)
      tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
      tokenizer.pad_token = tokenizer.eos_token

      model = PeftModel.from_pretrained(model, PEFT_MODEL)
      return model ,tokenizer

In [22]:
def endOfresponse(string):
    return string.find("[")

In [23]:
def prediction(prompt,model,tokenizer):
    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")

    with torch.inference_mode():
        output = model.generate(input_ids=encoding.input_ids,
                                attention_mask=encoding.attention_mask,
                                max_new_tokens=256,
                                temperature=0.000001,pad_token_id=tokenizer.eos_token_id,
                                eos_token_id=tokenizer.eos_token_id)

    response=tokenizer.decode(output[0], skip_special_tokens=True)
    index=endOfresponse(response[len(prompt)-4:])
    print(response)
    if index!= -1:
            return response[len(prompt)-4:len(prompt)-4+index]
    return response[len(prompt)-4:]
    

    
   

In [28]:
model ,tokenizer=LoadModel1('zaroui/vicuna13B')

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [35]:
prompt='<s> [INST] <<SYS>> Answer this question <</SYS>> '+'cool@c'+ ' [/INST] ' #this format only for vicuna .for other models prompt= question+'\n'

In [36]:
prediction(prompt,model ,tokenizer)


''